In [12]:
import os
import time
import math
import warnings
from typing import Iterable, Tuple, Dict, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from shapely.geometry import Point, MultiPoint, Polygon, MultiPolygon, LineString
from shapely.ops import unary_union, polygonize
from shapely.validation import make_valid

import geopandas as gpd
import contextily as cx
import alphashape
from scipy.spatial import Delaunay, KDTree
from tqdm import tqdm
import matplotlib.cm as cm
import matplotlib.colors as  mcolors
import folium
warnings.filterwarnings("ignore")
from IPython.display import display
import random
from utils import *
from alpha_algos import *

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# -----------------------------
# LIST OF LOCAL AUTHORITIES
# -----------------------------
local_auth = [
    "S12000005", "S12000006", "S12000008", "S12000009", "S12000010",
    "S12000011", "S12000013", "S12000014", "S12000015", "S12000017",
    "S12000018", "S12000019", "S12000020", "S12000021", "S12000023",
    "S12000024", "S12000026", "S12000027", "S12000028", "S12000029",
    "S12000030", "S12000033", "S12000034", "S12000035", "S12000036",
    "S12000038", "S12000039", "S12000040", "S12000041", "S12000042",
    "S12000043", "S12000044"
]

In [4]:
# -----------------------------
# FUNCTION TO LOAD DATA
# -----------------------------
def load_data(file_name="Accidents_2014.csv"):
    """
    Loads UK accident data and returns a cleaned DataFrame
    with only Longitude, Latitude, and Local Authority columns.
    Drops missing or implausible coordinates.
    """
    print("Loading data…")
    
    # Load CSV
    df = pd.read_csv(file_name)
    
    # Ensure required columns exist
    needed = ["Latitude", "Longitude", "Local_Authority_(Highway)"]
    for col in needed:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")
    
    # Drop rows with missing values
    df = df.dropna(subset=needed).copy()
    
    # Keep plausible UK coordinates
    df = df[
        (df["Latitude"].between(49.5, 61.5)) &
        (df["Longitude"].between(-8.5, 2.0))
    ]
    
    regions = df["Local_Authority_(Highway)"].unique()
    print(f"Regions found: {len(regions)}")
    
    # Keep only necessary columns
    df = df[["Longitude", "Latitude", "Local_Authority_(Highway)"]]
    
    return df

In [5]:
def map_plot(gdf):
    # Get bounding box of all geometries
    x1, y1, x2, y2 = gdf.total_bounds

    # Initialize Folium map centered on bounding box
    m = folium.Map(tiles="openstreetmap")
    m.fit_bounds([[y1, x1], [y2, x2]])

    # Assign unique colors to polygons using Matplotlib colormap
    colormap = cm.get_cmap("tab20", len(gdf))
    gdf["color"] = [mcolors.rgb2hex(colormap(i)) for i in range(len(gdf))]

    # Add polygons to Folium map with individual colors
    folium.GeoJson(
        gdf,
        style_function=lambda feature: {
            "fillColor": feature["properties"]["color"],
            "color": feature["properties"]["color"],
            "weight": 2,
            "fillOpacity": 0.6,
        },
    ).add_to(m)
    return(m)

In [13]:
"""
Main Workflow for Generating Alpha Shapes per Local Authority
and Visualizing Them on a Folium Map
"""


# -----------------------------
# MAIN WORKFLOW
# -----------------------------
def main():
    # Load raw data (user-defined function)
    df = load_data()

    # Store polygons generated for each local authority
    geo_arr = []

    # Process each local authority separately
    for location in local_auth:
        # Filter dataset for the current local authority
        df_cluster = df[df["Local_Authority_(Highway)"] == location]

        # Convert to list of (lon, lat) tuples
        points_list = [
            (float(lon), float(lat))
            for lon, lat in zip(df_cluster["Longitude"], df_cluster["Latitude"])
        ]

        # Determine optimal alpha parameter 
        alpha = optimized_alpha(points_list)

        # Generate polygon using alphashape 
        geom = alphashape_safe(points_list, alpha)

        # Append generated polygon to the list
        geo_arr.append(geom)

    # Create GeoDataFrame from list of polygons
    gdf = gpd.GeoDataFrame(geometry=geo_arr, crs="EPSG:4326")

    m =map_plot(gdf) 

    # Display interactive map (works in Jupyter/IPython)
    display(m)
    #-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
    #-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
    #-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
    
    #convex hull ---> alpha =0 
    
    for location in local_auth:
        # Filter dataset for the current local authority
        df_cluster = df[df["Local_Authority_(Highway)"] == location]

        # Convert to list of (lon, lat) tuples
        points_list = [
            (float(lon), float(lat))
            for lon, lat in zip(df_cluster["Longitude"], df_cluster["Latitude"])
        ]


        # Generate polygon using concave hull
        geom = alphashape_safe(points_list, 0)

        # Append generated polygon to the list
        geo_arr.append(geom)

    # Create GeoDataFrame from list of polygons
    gdf = gpd.GeoDataFrame(geometry=geo_arr, crs="EPSG:4326")
    m = map_plot(gdf)
    
    # Display interactive map (works in Jupyter/IPython)
    display(m)


# -----------------------------
# SCRIPT ENTRY POINT
# -----------------------------
if __name__ == "__main__":
    main()


Loading data…
Regions found: 207
